In [1]:
import numpy as np
import os
from os.path import isfile
import keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense, TimeDistributed, LSTM, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, Flatten, Conv2D, BatchNormalization, Lambda
from keras.layers.advanced_activations import ELU
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras import backend
from keras.utils import np_utils
from keras.optimizers import Adam, RMSprop

from keras import regularizers


import random

import torch
from   torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from   torchvision import datasets, transforms
import torchvision.models as models

import librosa
import librosa.display
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
dict_genres = {'Electronic':0, 'Experimental':1, 'Folk':2, 'Hip-Hop':3, 
               'Instrumental':4,'International':5, 'Pop' :6, 'Rock': 7  }


reverse_map = {v: k for k, v in dict_genres.items()}
print(reverse_map)

{0: 'Electronic', 1: 'Experimental', 2: 'Folk', 3: 'Hip-Hop', 4: 'Instrumental', 5: 'International', 6: 'Pop', 7: 'Rock'}


In [3]:
npzfile = np.load('shuffled_train.npz')
print(npzfile.files)
X_train = npzfile['arr_0']
y_train = npzfile['arr_1']
print(X_train.shape, y_train.shape)


['arr_0', 'arr_1']
(6394, 640, 128) (6394,)


In [4]:
npzfile = np.load('shuffled_valid.npz')
print(npzfile.files)
X_valid = npzfile['arr_0']
y_valid = npzfile['arr_1']
print(X_valid.shape, y_valid.shape)

['arr_0', 'arr_1']
(800, 640, 128) (800,)


In [5]:
npzfile = np.load('shuffled_test.npz')
print(npzfile.files)
X_test = npzfile['arr_0']
y_test = npzfile['arr_1']
print(X_valid.shape, y_valid.shape)

['arr_0', 'arr_1']
(800, 640, 128) (800,)


In [6]:
train_X = torch.from_numpy(X_train).type(torch.Tensor)
dev_X = torch.from_numpy(X_valid).type(torch.Tensor)
test_X = torch.from_numpy(X_test).type(torch.Tensor)

# Targets is a long tensor of size (N,) which tells the true class of the sample.
train_Y = torch.from_numpy(y_train).type(torch.LongTensor)
dev_Y = torch.from_numpy(y_valid).type(torch.LongTensor)
test_Y = torch.from_numpy(y_test).type(torch.LongTensor)

In [105]:
from PIL import Image
xtrain = torch.Tensor()
compose = transforms.Compose([transforms.ToPILImage(),transforms.Resize((128,128),interpolation=Image.NEAREST)])
compose1 = transforms.Compose([transforms.ToTensor()])
train_X1 = train_X.unsqueeze(1).cpu()
for imgdata in train_X1:
    a_trans = compose(imgdata)
    a_trans = compose1(a_trans)
    xtrain = torch.cat((xtrain, a_trans),0)


AttributeError: 'Tensor' object has no attribute 'usqueeze'

In [108]:
xtrain = xtrain.unsqueeze(1)
xtrain.shape

torch.Size([6394, 1, 128, 128])

In [109]:
xval = torch.Tensor()
val_X1 = dev_X.unsqueeze(1).cpu()
for imgdata in val_X1:
    a_trans = compose(imgdata)
    a_trans = compose1(a_trans)
    xval = torch.cat((xval, a_trans),0)
xval = xval.unsqueeze(1)

In [110]:
xtest = torch.Tensor()
test_X1 = test_X.unsqueeze(1).cpu()
for imgdata in test_X1:
    a_trans = compose(imgdata)
    a_trans = compose1(a_trans)
    xtest = torch.cat((xtest, a_trans),0)
xtest = xtest.unsqueeze(1)

In [116]:
batch_size = 32*2*2
num_classes = 8
n_features = X_train.shape[2]
n_time = X_train.shape[1]

In [8]:
class cnnrnn(nn.Module):
    def __init__(self, embed_size = 128, num_layers = 4, output_dim = 8, batchsize = 100):
        super(cnnrnn,self).__init__()
       
        '''Encoder'''
       
        # conv1d layers
        self.conv1 = nn.Sequential(         
            nn.Conv2d(1, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 64 * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.MaxPool2d(kernel_size=(2, 2)),
            nn.Conv2d(64 * 2, 64 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.MaxPool2d(kernel_size=(2, 2)),
            nn.Conv2d(64 * 4, 64 * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(p=0.5),
            #nn.MaxPool2d(kernel_size=(2, 2)),
            nn.Conv2d(64 * 8, 64*16, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(p=0.5),
            #nn.MaxPool2d(kernel_size=(2, 2)),
            nn.Flatten(),
            nn.Linear(64*16,128),
            nn.Sigmoid()
        )
        
        

        # add another fully connected layer
        self.embed = nn.Linear(in_features=128, out_features=128)
       
        # dropout layer
        self.dropout = nn.Dropout(p=0.5)
       
        # activation layers
        self.prelu = nn.PReLU()
       
        '''Decoder'''
       
        self.hidden_dim1 = 128
        self.hidden_dim2 = 64
        self.num_layers = num_layers
        self.output_dim = output_dim
        self.batch_size = batchsize
       
        # LSTM layer
        self.lstm = nn.LSTM(embed_size, self.hidden_dim1, self.num_layers)

        # output layer
        self.linear1 = nn.Linear(self.hidden_dim1, self.hidden_dim2)
       
        # output layer
        self.linear2 = nn.Linear(self.hidden_dim2, self.output_dim)
       
        # dropout layer
        self.dropout1 = nn.Dropout(p=0.4)
   
    def init_hidden(self):
        return (
            torch.zeros(self.num_layers, self.batch_size, self.hidden_dim1),
            torch.zeros(self.num_layers, self.batch_size, self.hidden_dim1),
        )
       
    def cnnencoder(self, images):
        x = self.conv1(images)
        
        # get the embeddings from the densenet
        convnet_outputs = self.dropout(self.prelu(images))
       
        # pass through the fully connected
        embeddings = self.embed(convnet_outputs)
       
        return embeddings
   
    def rnndecoder(self, features):
        lstm_out, hidden = self.lstm(features.squeeze())
        logits1 = self.dropout1(self.linear1(lstm_out[-1]))
        logits = self.linear2(logits1)
        genre_scores = F.log_softmax(logits, dim=1)
        return genre_scores
   
    def forward(self, inputs):
        embed = self.cnnencoder(inputs)
        output = self.rnndecoder(embed)
        return output
   
    def get_accuracy(self, logits, target):
        """ compute accuracy for training round """
        corrects = (
            torch.max(logits, 1)[1].view(target.size()).data == target.data
        ).sum()
        accuracy = 100.0 * corrects / self.batch_size
        return accuracy.item()

In [205]:
model = cnnrnn(embed_size = 128, num_layers = 2, output_dim = 8, batchsize = batch_size).cuda()
loss_function = nn.NLLLoss()  # expects ouputs from LogSoftmax

optimizer = optim.Adam(model.parameters(), lr=0.001)
train_on_gpu = torch.cuda.is_available()
if train_on_gpu:
    print("\nTraining on GPU")
else:
    print("\nNo GPU, training on CPU")


Training on GPU


In [206]:
# all training data (epoch) / batch_size == num_batches (12)
num_batches = int(train_X.shape[0] / batch_size)
num_dev_batches = int(dev_X.shape[0] / batch_size)

In [207]:
num_batches

49

In [208]:
val_loss_list, val_accuracy_list, epoch_list = [], [], []

In [209]:
xtrain = xtrain.cuda()
train_Y = train_Y.cuda()
xval = xval.cuda()
dev_Y = dev_Y.cuda()
xtest = xtest.cuda()
test_Y = test_Y.cuda()


In [ ]:
model.train()
for epoch in range(2000):

    train_running_loss, train_acc = 0.0, 0.0

    # Init hidden state - if you don't want a stateful LSTM (between epochs)
    model.hidden = model.init_hidden()
    
    for i in range(num_batches):

        # zero out gradient, so they don't accumulate btw epochs
        model.zero_grad()

        # train_X shape: (total # of training examples, sequence_length, input_dim)
        # train_Y shape: (total # of training examples, # output classes)
        #
        # Slice out local minibatches & labels => Note that we *permute* the local minibatch to
        # match the PyTorch expected input tensor format of (sequence_length, batch size, input_dim)
        X_local_minibatch, y_local_minibatch = (
            xtrain[i * batch_size : (i + 1) * batch_size,],
            train_Y[i * batch_size : (i + 1) * batch_size,],
        )

        # Reshape input & targets to "match" what the loss_function wants
        #X_local_minibatch1 = X_local_minibatch

        # NLLLoss does not expect a one-hot encoded vector as the target, but class indices
        #y_local_minibatch = torch.max(y_local_minibatch)

        y_pred = model(X_local_minibatch)                # fwd the bass (forward pass)
        loss = loss_function(y_pred, y_local_minibatch)  # compute loss
        loss.backward()                                  # reeeeewind (backward pass)
        optimizer.step()                                 # parameter update

        train_running_loss += loss.detach().item()       # unpacks the tensor into a scalar value
        train_acc += model.get_accuracy(y_pred, y_local_minibatch)

    print(
        "Epoch:  %d | NLLoss: %.4f | Train Accuracy: %.2f"
        % (epoch, train_running_loss / num_batches, train_acc / num_batches)
    )

    print("Validation ...")  # should this be done every N epochs
    
    if epoch % 10 == 0:
        val_running_loss, val_acc = 0.0, 0.0

        # Compute validation loss, accuracy. Use torch.no_grad() & model.eval()
        with torch.no_grad():
            model.eval()

            model.hidden = model.init_hidden()
            for i in range(num_dev_batches):
                X_local_validation_minibatch, y_local_validation_minibatch = (
                    xval[i * batch_size : (i + 1) * batch_size,],
                    dev_Y[i * batch_size : (i + 1) * batch_size,],
                )
                #X_local_minibatch = X_local_validation_minibatch.permute(1, 0, 2)
                #y_local_minibatch = torch.max(y_local_validation_minibatch)

                y_pred = model(X_local_minibatch)
                val_loss = loss_function(y_pred, y_local_minibatch)

                val_running_loss += (
                    val_loss.detach().item()
                )  # unpacks the tensor into a scalar value
                val_acc += model.get_accuracy(y_pred, y_local_minibatch)

            model.train()  # reset to train mode after iterationg through validation data
            print(
                "Epoch:  %d | NLLoss: %.4f | Train Accuracy: %.2f | Val Loss %.4f  | Val Accuracy: %.2f"
                % (
                    epoch,
                    train_running_loss / num_batches,
                    train_acc / num_batches,
                    val_running_loss / num_dev_batches,
                    val_acc / num_dev_batches,
                )
            )

        epoch_list.append(epoch)
        val_accuracy_list.append(val_acc / num_dev_batches)
        val_loss_list.append(val_running_loss / num_dev_batches)

Epoch:  0 | NLLoss: 2.0827 | Train Accuracy: 11.78
Validation ...
Epoch:  0 | NLLoss: 2.0827 | Train Accuracy: 11.78 | Val Loss 2.0865  | Val Accuracy: 6.00
Epoch:  1 | NLLoss: 2.0814 | Train Accuracy: 11.67
Validation ...
Epoch:  2 | NLLoss: 2.0802 | Train Accuracy: 12.08
Validation ...
Epoch:  3 | NLLoss: 2.0805 | Train Accuracy: 12.12
Validation ...
Epoch:  4 | NLLoss: 2.0805 | Train Accuracy: 11.41
Validation ...
Epoch:  5 | NLLoss: 2.0806 | Train Accuracy: 11.43
Validation ...
Epoch:  6 | NLLoss: 2.0797 | Train Accuracy: 12.10
Validation ...
Epoch:  7 | NLLoss: 2.0800 | Train Accuracy: 11.78
Validation ...
Epoch:  8 | NLLoss: 2.0799 | Train Accuracy: 12.35
Validation ...
Epoch:  9 | NLLoss: 2.0796 | Train Accuracy: 12.53
Validation ...
Epoch:  10 | NLLoss: 2.0801 | Train Accuracy: 12.06
Validation ...
Epoch:  10 | NLLoss: 2.0801 | Train Accuracy: 12.06 | Val Loss 2.0809  | Val Accuracy: 14.00
Epoch:  11 | NLLoss: 2.0802 | Train Accuracy: 11.90
Validation ...
Epoch:  12 | NLLoss: 2

Epoch:  107 | NLLoss: 2.0511 | Train Accuracy: 16.92
Validation ...
Epoch:  108 | NLLoss: 2.0515 | Train Accuracy: 17.29
Validation ...
Epoch:  109 | NLLoss: 2.0490 | Train Accuracy: 17.12
Validation ...
Epoch:  110 | NLLoss: 2.0452 | Train Accuracy: 17.35
Validation ...
Epoch:  110 | NLLoss: 2.0452 | Train Accuracy: 17.35 | Val Loss 1.9517  | Val Accuracy: 34.00
Epoch:  111 | NLLoss: 2.0447 | Train Accuracy: 16.88
Validation ...
Epoch:  112 | NLLoss: 2.0467 | Train Accuracy: 17.45
Validation ...
Epoch:  113 | NLLoss: 2.0514 | Train Accuracy: 16.76
Validation ...
Epoch:  114 | NLLoss: 2.0511 | Train Accuracy: 16.73
Validation ...
Epoch:  115 | NLLoss: 2.0456 | Train Accuracy: 17.71
Validation ...
Epoch:  116 | NLLoss: 2.0502 | Train Accuracy: 17.71
Validation ...
Epoch:  117 | NLLoss: 2.0439 | Train Accuracy: 17.45
Validation ...
Epoch:  118 | NLLoss: 2.0465 | Train Accuracy: 17.65
Validation ...
Epoch:  119 | NLLoss: 2.0430 | Train Accuracy: 17.90
Validation ...
Epoch:  120 | NLLoss: 

In [193]:
    model.train()
    for i in range(num_batches):

        # zero out gradient, so they don't accumulate btw epochs
        model.zero_grad()

        # train_X shape: (total # of training examples, sequence_length, input_dim)
        # train_Y shape: (total # of training examples, # output classes)
        #
        # Slice out local minibatches & labels => Note that we *permute* the local minibatch to
        # match the PyTorch expected input tensor format of (sequence_length, batch size, input_dim)
        X_local_minibatch, y_local_minibatch = (
            xtrain[i * batch_size : (i + 1) * batch_size,],
            train_Y[i * batch_size : (i + 1) * batch_size,],
        )

        # Reshape input & targets to "match" what the loss_function wants
        #X_local_minibatch1 = X_local_minibatch

        # NLLLoss does not expect a one-hot encoded vector as the target, but class indices
        #y_local_minibatch = torch.max(y_local_minibatch)

        y_pred = model(X_local_minibatch)                # fwd the bass (forward pass)
        loss = loss_function(y_pred, y_local_minibatch)  # compute loss
        loss.backward()                                  # reeeeewind (backward pass)
        optimizer.step()                                 # parameter update

        train_running_loss += loss.detach().item()       # unpacks the tensor into a scalar value
        train_acc += model.get_accuracy(y_pred, y_local_minibatch)

In [195]:
train_acc

1203.0

In [114]:
image_vectors.shape

torch.Size([128, 8])

In [115]:
# add another fully connected layer
embed = nn.Linear(in_features=128, out_features=128).cuda()
# dropout layer
dropout = nn.Dropout(p=0.5).cuda()
# activation layers
prelu = nn.PReLU().cuda()

In [18]:
image_vectors = conv1(images)

RuntimeError: size mismatch, m1: [128 x 5120], m2: [1024 x 128] at C:/w/1/s/tmp_conda_3.7_055457/conda/conda-bld/pytorch_1565416617654/work/aten/src\THC/generic/THCTensorMathBlas.cu:273

In [203]:
# get the embeddings from the densenet
convnet_outputs = dropout(prelu(image_vectors))
       
# pass through the fully connected
embeddings = embed(convnet_outputs)

In [102]:
da = torch.Tensor()

In [73]:
compose = transforms.Compose([transforms.ToPILImage(),transforms.Resize((128,128),interpolation=Image.NEAREST)])
compose1 = transforms.Compose([transforms.ToTensor()])
#a_trans = compose(train_X.unsqueeze(1)[5].cpu())

In [62]:
from PIL import Image

KeyboardInterrupt: 

In [93]:
da= torch.cat((a_trans.squeeze(1), a_trans.squeeze(1)),0)

In [104]:
da.shape

torch.Size([462, 128, 128])

In [155]:
#fashiontest

In [163]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy as np
import torch
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch import nn
from torch import optim
import torch.nn.functional as F
from collections import OrderedDict
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize([0.5],[0.5]),
                             ])
# Download and load the training data
trainset = datasets.FashionMNIST('F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
testset = datasets.FashionMNIST('F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)


In [164]:
for e in range(1):
    running_loss = 0
    for images, labels in iter(trainloader):
        steps += 1
        # Flatten MNIST images into a 784 long vector
        images.resize_(images.size()[0], 784)

NameError: name 'steps' is not defined

In [169]:
input_size = 784
hidden_sizes = [200, 100]
output_size = 10
model = nn.Sequential(OrderedDict([
                      ('fc1', nn.Linear(input_size, hidden_sizes[0])),
                      ('relu1', nn.ReLU()),
                      ('fc2', nn.Linear(hidden_sizes[0], hidden_sizes[1])),
                      ('relu2', nn.ReLU()),
                      ('output', nn.Linear(hidden_sizes[1], output_size)),
                      ('relu3', nn.ReLU())]))

    
# Traning Our Model
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [173]:
# Traning Our Model
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

images, labels = next(iter(trainloader))
images.resize_(64, 784)

# Clear the gradients, do this because gradients are accumulated
optimizer.zero_grad()

# Forward pass, then backward pass, then update weights
output = model.forward(images)
loss = criterion(output, labels)
loss.backward()
#print('Gradient -', model.fc1.weight.grad)
optimizer.step()

In [175]:
output.shape

torch.Size([64, 10])